In [1]:
# Store environmental variable
from getpass import getpass
password = getpass('Enter database password')
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://dataviz.c8bvxdjysbw2.us-east-1.rds.amazonaws.com:5432/app_reviews"
config = {"user":"postgres",
          "password": password,
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [2]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-02-15 08:15:39--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2022-02-15 08:15:40 (5.84 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16_Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Mobile_Electronics_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
mobile_app_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Mobile_Electronics_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
mobile_app_df.show()

mobile_app_df.printSchema()

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   20422322| R8MEA6IGAHO0B|B00MC4CED8|     217304173|BlackVue DR600GW-PMP|Mobile_Electronics|          5|            0|          0|   N|                Y|         Very Happy!|As advertised. Ev...| 2015-08-31|
|         US|   40835037|R31LOQ8JGLPRLK|B00OQMFG1Q|     137313254|GENSSI GSM / GPS ...|Mobile_Electronics|      

In [6]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
# ^^ 

In [7]:
# Create the customers_table DataFrame
#customers_df = df.groupby("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "customer_count")
#customers_df.show()

In [7]:
#1.) #Create a total_votes df
total_votes_df = mobile_app_df.filter("total_votes>=20")
total_votes_df.show()



+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48701722| RBEMQ29WJBHYG|B00YO3UYXW|     913911909|Jensen MCR-100 Ca...|Mobile_Electronics|          5|          164|        168|   N|                Y|  I LOVE my recorder|I LOVE my recorde...| 2015-08-31|
|         US|   40963441|R2JK5Y8D5MXAGP|B00SU7B0JK|      82850235|iXCC Multi pack L...|Mobile_Electronics|      

In [8]:
#2.) # Create a DataFrame to retrieve all rows of helpful_votes divided by total_votes is equal to or greater than 50%.
helpful_votes_df = total_votes_df.filter("helpful_votes/total_votes>=.50")
helpful_votes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48701722| RBEMQ29WJBHYG|B00YO3UYXW|     913911909|Jensen MCR-100 Ca...|Mobile_Electronics|          5|          164|        168|   N|                Y|  I LOVE my recorder|I LOVE my recorde...| 2015-08-31|
|         US|   25750755|R13W3EMIUV120L|B00YO3UYXW|     913911909|Jensen MCR-100 Ca...|Mobile_Electronics|      

In [9]:
#
print(helpful_votes_df.count())

1068


In [10]:
vine_review_df = helpful_votes_df.filter(helpful_votes_df["vine"] == "Y")
vine_review_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   52490988|R3BOQTLUU3Y4L8|B00EP0SIOG|     628459233|Orbotix Sphero 2....|Mobile_Electronics|          4|           42|         55|   Y|                N|Lot of fun, but f...|Before receiving ...| 2013-11-19|
|         US|   48814959| RWF03LXVXC22A|B00EP0SIOG|     628459233|Orbotix Sphero 2....|Mobile_Electronics|      

In [11]:
no_vine_review_df = helpful_votes_df.filter(helpful_votes_df["vine"] == "N")
no_vine_review_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48701722| RBEMQ29WJBHYG|B00YO3UYXW|     913911909|Jensen MCR-100 Ca...|Mobile_Electronics|          5|          164|        168|   N|                Y|  I LOVE my recorder|I LOVE my recorde...| 2015-08-31|
|         US|   25750755|R13W3EMIUV120L|B00YO3UYXW|     913911909|Jensen MCR-100 Ca...|Mobile_Electronics|      

In [12]:
from pyspark.sql.functions import count

#5-1
#Total number of reviews for each df
vine_review_count = vine_review_df.count()
print("Total Number of Reviews paid/ 'helpful': %f" % vine_review_count)

no_vine_review_count = no_vine_review_df.count()
print("Total Number of Reviews non-paid/ 'helpful': %f" % no_vine_review_count)


Total Number of Reviews paid/ 'helpful': 4.000000
Total Number of Reviews non-paid/ 'helpful': 1064.000000


In [13]:
#5-2
#Number of 5star reviews for above paid and non paid df's
five_star_vine_df = vine_review_df.filter(no_vine_review_df["star_rating"]=="5")
print("Total Number of 5 star reviews paid/ 'helpful': %f" % five_star_vine_df.count())

five_star_novine_df = no_vine_review_df.filter(no_vine_review_df["star_rating"]=="5")
print("Total Number of 5 star reviews nonpaid/ 'helpful': %f" % five_star_novine_df.count())


Total Number of 5 star reviews paid/ 'helpful': 1.000000
Total Number of 5 star reviews nonpaid/ 'helpful': 527.000000


In [14]:
#5-3
#Percentage of 5 star reviews for paid / nonpaid dfs
five_star_rev = (five_star_vine_df.count()/vine_review_count)
print("Percent of 5 star app reviews from paid/, 'helpful' dataset: %f" % five_star_rev)

five_star_rev_no_vine = (five_star_novine_df.count()/no_vine_review_count)
print("Percent of 5 star app reviews from nonpaid/, 'helpful' dataset: %f" % five_star_rev_no_vine)

Percent of 5 star app reviews from paid/, 'helpful' dataset: 0.250000
Percent of 5 star app reviews from nonpaid/, 'helpful' dataset: 0.495301


In [15]:
#Total number of all reviews
total_reviews_count = mobile_app_df.count()
print("Total number of app reviews: %f" % total_reviews_count)

#Total number of all 'helpful' reviews
help_reviews_count = helpful_votes_df.count()
print("Total number of 'helpful' reviews: %f" % help_reviews_count)

Total number of app reviews: 104975.000000
Total number of 'helpful' reviews: 1068.000000


In [24]:
#Total number of all five star reviews
five_star_df = mobile_app_df.filter(mobile_app_df["star_rating"] =='5')
#five_star_df.show()

five_star_count = five_star_df.count()
print("Total number of five star app reviews: %f" % five_star_count)

Total number of five star app reviews: 52255.000000


In [25]:
#Total number of five star helpful reviews
#5-4
five_star_helpful_df = helpful_votes_df.filter(helpful_votes_df["star_rating"]== '5')
#five_star_helpful_df.show()

In [18]:
five_star_helpful_count = five_star_helpful_df.count()
print("Total number of five star 'helpful' app reviews: %f" % five_star_helpful_count)

Total number of five star 'helpful' app reviews: 528.000000


In [19]:
five_star_vine_all_df = five_star_df.filter(five_star_df["vine"] == "Y")
five_star_vine_all_count = five_star_vine_all_df.count()
#vine_five_star_df.show()

In [20]:
five_star_vine_all_pct = (five_star_vine_all_count/five_star_count)
print("Number of five star reviews : %f" % five_star_vine_all_count)
print("Percent of five star reviews : %f" % five_star_vine_all_pct)

Number of five star reviews : 6.000000
Percent of five star reviews : 0.000115


In [21]:
#Five star nonpaid reviews
five_star_no_vine_df = five_star_df.filter(five_star_df["vine"] == "N")
five_star_no_vine_count = five_star_no_vine_df.count()

In [22]:
five_star_no_vine_all_pct = five_star_no_vine_count/ five_star_count
print("Number of five star reviews 'nonpaid' : %f" % five_star_no_vine_count)
print("Percent of five star reviews 'nonpaid' : %f" % five_star_no_vine_all_pct)

Number of five star reviews 'non' : 52249.000000
Percent of five star reviews 'non' : 0.999885


In [23]:
#percent of helpful five star reviews
vine_five_star_helpful_df = five_star_helpful_df.filter(five_star_helpful_df["vine"] =="Y")
vine_five_star_helpful_count = vine_five_star_helpful_df.count()

vine_five_star_pct = (vine_five_star_helpful_count/ five_star_helpful_count)
print("Number of five star, 'helpful' app reviews : %f" % vine_five_star_helpful_count)
print("Percent of five star, 'helpful' app reviews : %f" % vine_five_star_pct)

Number of five star, 'helpful' app reviews : 1.000000
Percent of five star, 'helpful' app reviews : 0.001894
